<a href="https://colab.research.google.com/github/noambassat/Rag_on_arxiv/blob/main/RAG_with_arxiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
import re

In [86]:
"https://arxiv.org/list/cs.HC/" + datetime(2025,3,26).strftime("%Y-%m")

'https://arxiv.org/list/cs.HC/2025-03'

In [87]:
def get_url(date):
  path = "https://arxiv.org/list/cs.HC/" + date.strftime("%Y-%m")
  content = requests.get(path).text
  return [a["href"] for a in bs(content, "html.parser").find_all("a",{"title":"View HTML"})]

In [88]:
# urls = sum([get_url(datetime(2025, 3, 26)-timedelta(days=i*30))
#         for i in trange(5)],[])

urls = get_url(datetime(2025, 3, 26))
print(urls[0])

https://arxiv.org/html/2503.00144v1


In [89]:
def get_article(url):
    content = requests.get(url).text
    article = bs(content, "html.parser").find("div",{"class":"ltx_page_content"})
    return [s.text for s in article.find_all("section")]

In [90]:
articles = pd.DataFrame(
    {
        "url":urls,
        "article":[get_article(url) for url in tqdm(urls)]
    }
)



 26%|██▌       | 12/47 [06:11<18:02, 30.92s/it]

100%|██████████| 47/47 [00:15<00:00,  3.07it/s]


In [91]:
articles["article"].isna().sum()

np.int64(0)

In [98]:
articles.shape

(47, 2)

In [97]:
articles.head()

,url,article
0,https://arxiv.org/html/2503.00144v1,[\n\n1 Introduction\n\nProgramming is an impor...
1,https://arxiv.org/html/2503.00149v2,[\n\n1. Introduction\n\nTactile charts are an ...
2,https://arxiv.org/html/2503.00228v1,[\n\n1. Introduction\n\nSimilarity is a fundam...
3,https://arxiv.org/html/2503.00257v1,[\n\n1. Introduction\n\nThe Japanese manga111M...
4,https://arxiv.org/html/2503.00303v1,[\n\nI Introduction\n\n\nThis document is a mo...


In [100]:
paragraphs = articles.explode('article').rename(columns={"article":"paragraph"}) # for each url there are several paragraph
paragraphs = paragraphs[paragraphs['paragraph'].str.split().map(len)>10]
paragraphs.head()

,url,paragraph
0,https://arxiv.org/html/2503.00144v1,\n\n1 Introduction\n\nProgramming is an import...
0,https://arxiv.org/html/2503.00144v1,\n\n2 Related Work\n\n\n2.0.1 Learner-System C...
0,https://arxiv.org/html/2503.00144v1,\n\n2.0.1 Learner-System Control Dynamics.\n\n...
0,https://arxiv.org/html/2503.00144v1,\n\n2.0.2 Participatory Design in Educational ...
0,https://arxiv.org/html/2503.00144v1,\n\n3 Participatory Design with Learners and I...


In [101]:
paragraphs.shape

(1695, 2)


# Embeddings

In [108]:
pip install --upgrade openai

In [121]:
from openai import OpenAI
from google.colab import userdata
import os

In [125]:
open_ai_key=userdata.get('open_ai_key')
os.environ['OPENAI_API_KEY'] = open_ai_key
client = OpenAI()
# client.api_key =
def get_embedding(text, model='text-embedding-ada-002'):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [127]:
len(get_embedding(paragraphs["paragraph"].iloc[0]))

1536

In [ ]:
tqdm.pandas()
paragraphs["embeddings"] = paragraphs["paragraph"].progress_apply(get_embedding)

  6%|▌         | 98/1695 [00:34<13:41,  1.94it/s]